# Getting Started with Hadoop/Python
- - -

In this tutorial, students will learn how to use Python with Apache Hadoop to store, process, and analyze incredibly large data sets. Hadoop has become the standard in distributed data processing, but has mostly required Java in the past. Today, there are a numerous open source projects that support Hadoop in Python and this tutorial will show students how to use them.

Working with Hadoop using Python instead of Java is entirely possible with a conglomeration of active open source projects that provide Python APIs to Hadoop components. This tutorial will survey the most important projects and show that not only is Hadoop with Python possible, but that it also has some advantages over Hadoop with Java.

The reasons for using Hadoop with Python instead of Java are not all that different than the classic Java vs. Python arguments. One of the most important differences is not having to compile your code by instead using a scripting language. This makes more interactive development of analytics possible, makes maintaining and fixing applications in production environments simpler in many cases, makes for more succinct and easier to read code, and so much more. Also, by integrating Python with Hadoop, you get access to the world-class data analysis libraries such as numpy, scipy, nltk, and scikit-learn that are best-in-breed both inside of Python and outside.

Students will be surprised at how quickly they can get up and running with Hadoop when using Python. In this tutorial, we will talk about the following libraries and approaches and will guide students through a series of exercises.

In [6]:
#!/usr/bin/env python3

import pandas as pd
import re
import os
import PyPDF2
from snakebite.client import Client

# if using jupyterhub, start a session using the "Python 3 + PySpark" kernel
from pyspark import SparkContext, SparkConf


# step 1: load file from HDFS
hdfs = Client("localhost", 9000, use_trash=False)

# get a full object
pdf = hdfs.cat(['/user/ops_srashid/origin_of_species.pdf'])

# convert pdf to text
pdfObject = PyPDF2.PdfFileReader(pdf)
wordlist = pdfObject.extractText()
hdfs.mkdir('/user/ops_srashid/extract')


# step 2: normalize data

# replace empty lines, punctuation, and special characters with space
# convert all words to lowercase
words = re.compile(r'\W+', re.UNICODE).split(wordlist.lower())
# print(sorted(words))


# step 3: save normalized, intermediate data

# df: dataframe
df = pd.Series(sorted(words))
# df = np.array(sorted(words))

# f: file
f = '/tmp/normalized.pkl'

df.to_pickle(f)

# fd: file descriptor
fd = os.stat(f)


# step 4: create dictionary of words and frequencies
wordcount = Counter(df)

# quick statistic summary of your data
# print(wordcount.describe())


# step 5: output results
# print(wordcount)
# print(wordcount.index)
# print(wordcount.columns)
# print(wordcount.values)
# print(wordcount.sort_index(axis=1, ascending=False))
print(wordcount.most_common(10))

# step 6: do the same thing... but in Spark
# connect to spark
conf = SparkConf()
conf.setMaster('yarn-client')
conf.set('spark.executor.memory', "5g")
conf.set('spark.driver.memory', "500m")
conf.set('spark.yarn.dist.files','file:/usr/hdp/current/spark2-client/python/lib/pyspark.zip,file:/usr/hdp/current/spark2-client/python/lib/py4j-0.10.6-src.zip')
conf.setExecutorEnv('PYTHONPATH','pyspark.zip:py4j-0.10.6-src.zip')
sc = SparkContext(conf=conf)

# do spark work here
text_file = sc.textFile("hdfs://user/ops_srashid/extract/")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("hdfs://...")
# disconnect from spark
sc.stop()

SyntaxError: invalid syntax (client.py, line 1473)